In [1]:
import sys
sys.path.append('..')

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch  # PyTorch의 경우
torch.set_grad_enabled(False)
print(torch.cuda.device_count())  # GPU 개수 확인
print(torch.cuda.current_device())  # 현재 활성화된 GPU 확인

from transformers import AutoModelForCausalLM, AutoTokenizer

1
0


/home/kdkyum/.miniconda3/envs/engram/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = "meta-llama/Llama-2-13b-hf"
model_pretrain = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    use_cache=False,
)

local_model_path = "/home/kdkyum/workdir/llm_engram/model-output/Llama-2-13b-hf-bioS_multi5_permutes-lr5.0e-5/best_model"

model_finetune = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    use_cache=False,
)

model_pretrain.eval()
model_finetune.eval()

Loading checkpoint shards: 100%|██████████| 6/6 [00:07<00:00,  1.23s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((5120,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((5120,), eps=1e-05)
    (rotary_e

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Get the state dictionaries
state_dict_finetune = model_finetune.state_dict()
state_dict_pretrain = model_pretrain.state_dict()

# Compute differences for all parameters (flattened) in the shared keys
all_diffs = []
param_diffs = {}
for key, param in state_dict_finetune.items():
    if key in state_dict_pretrain:
        diff = torch.abs(param - state_dict_pretrain[key]).cpu().detach().flatten()
        all_diffs.append(diff)
        param_diffs[key] = diff

all_diffs = torch.cat(all_diffs)
threshold = torch.quantile(all_diffs, 0.99).item()
print("Threshold for top 1% changes:", threshold)

# Aggregate counts of top-1% changes per layer and per submodule (MLP and Attention)
layer_changes = {}  # { layer_id : {"attn": count, "mlp": count} }
for key, diff in param_diffs.items():
    # Only consider parameters that belong to a layer
    # Expected key format: "model.layers.{layer_id}.<submodule>...."
    if "model.layers." in key:
        parts = key.split(".")
        layer_id = parts[2]
        if layer_id not in layer_changes:
            layer_changes[layer_id] = {"attn": 0, "mlp": 0}
        # Classify whether the parameter is part of the MLP or Attention block.
        if "mlp" in key:
            layer_changes[layer_id]["mlp"] += (diff >= threshold).sum().item()
        elif "attn" in key:
            layer_changes[layer_id]["attn"] += (diff >= threshold).sum().item()

# Sort layer indices numerically
layers = sorted(layer_changes.keys(), key=lambda x: int(x))
attn_counts = [layer_changes[layer]["attn"] for layer in layers]
mlp_counts = [layer_changes[layer]["mlp"] for layer in layers]

# Plot the count of top 1% changed parameters per layer for Attention and MLP modules.
x = np.arange(len(layers))
width = 0.35
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(x - width/2, attn_counts, width, label='Attention')
ax.bar(x + width/2, mlp_counts, width, label='MLP')

ax.set_xlabel('Layer')
ax.set_ylabel('Count of top 1% changed parameters')
ax.set_title('Localization of Top 1% Most Changed Parameters per Layer')
ax.set_xticks(x)
ax.set_xticklabels(layers)
ax.legend()

plt.show()

RuntimeError: Tensor on device cuda:0 is not on the expected device meta!